In [8]:
import matplotlib.pyplot as plt
from src import logger
from src.utils.common import read_yaml, log_error, create_directory
from dataclasses import dataclass
from src.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
import cv2
import numpy as np
import os
from typing import List, Union, Optional
import sys
print(os.getcwd())


/Users/goldyrana/mess/deep_learning/projects/blood_group_detection


In [3]:
os.chdir("../")

In [14]:
@dataclass
class ImagePreprocessingConfig:
    raw_path: str
    interim_path: str

class ConfigurationManager:
    def __init__(self):
        self.config=read_yaml(CONFIG_FILE_PATH)
        self.params=read_yaml(PARAMS_FILE_PATH)
        print(self.config)
    def get_image_preprocessing_config(self):
        params=ImagePreprocessingConfig(
            raw_path=self.config.data.extract_to,
            interim_path=self.config.data.interim_path
        )
        return params


class ImagePreprocessing:
    def __init__(self, config):
        self.config=config

    @log_error(faliure_message="Image Preprocessing")
    def process_image(self, image):
        corrected_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        corrected_image = corrected_image.astype(np.uint8)

        clahe = cv2.createCLAHE(clipLimit=70.0, tileGridSize=(8, 8))
        enhanced_image = clahe.apply(corrected_image)
        
        # Apply erosion to thin and darken lines
        kernel = np.ones((1, 1), np.uint8)
        thinned_image = cv2.erode(enhanced_image, kernel, iterations=200)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(5,5))
        enhanced_contrast = clahe.apply(thinned_image)

        smoothed = cv2.GaussianBlur(enhanced_contrast, (3, 3), 23)
        # Step 3: Sharpen the image
        sharpened = cv2.addWeighted(enhanced_contrast,2, smoothed, -0.5, 0)

        denoised_image = cv2.fastNlMeansDenoising(sharpened, None, h=20, 
                                                templateWindowSize=5, 
                                                searchWindowSize=25)
        kernel = np.ones((1, 1), np.uint8)
        erode_image = cv2.erode(denoised_image, kernel, iterations=200)
        return erode_image
    
    @log_error(faliure_message="Reading Image")
    def read_image(self, path):
        image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        image = cv2.cvtColor(image).astype(np.uint8)
        return image
    
    @log_error(sucess_message=None, faliure_message="Get folder path")
    def get_folder_path(self, main_folder: str) -> List[str]:
        """
        To get sub folder names of given folder 
        Return:
        list: 
        """
        folder_names = list(os.listdir(main_folder)) # returns the folder name
        print(folder_names)
        folder_with_path = [] # to save foldername with base path
        
        for sub_folder in folder_names:
            if os.path.isdir(os.path.join(main_folder, sub_folder)) != True:
                logger.info(f"{sub_folder} not a directory\nRemoving {sub_folder}") # to remove reduant files 
            else:
                folder_with_path.append(os.path.join(main_folder, # base path
                                                     sub_folder)) # image_folder_name

        
        return folder_with_path

    @log_error(faliure_message="Failed in saving image")
    def save_image(self, image, save_path) -> None:
        create_directory(save_path, is_extension_present=True)
        cv2.imwrite(save_path, image)
        

    # def apply_image_operations(self, image_folders: List[str]):
    #     for image_folder in image_folders:
    #         images_names = os.listdir(image_folder)
            
    #         # process one image at time
    #         for image in images_names:
    #             # join image with base_path
    #             image_path = os.path.join(image_folder, image)
    #             pic = self.read_image(image_path)
    #             pic = self.process_image(pic)
    #             self.save_image(pic)
    #             pass


        

In [5]:
os.getcwd()

'/Users/goldyrana/mess/deep_learning/projects/blood_group_detection'

In [6]:
os.listdir("data/raw")

['.gitkeep', 'AB+', 'AB-', 'A+', 'B-', 'B+', 'A-', 'O-', 'O+']

In [18]:
folder_path


['data/raw/AB+',
 'data/raw/AB-',
 'data/raw/A+',
 'data/raw/B-',
 'data/raw/B+',
 'data/raw/A-',
 'data/raw/O-',
 'data/raw/O+']

In [15]:
if __name__ == "__main__":
    config_manager = ConfigurationManager()
    config_params = config_manager.get_image_preprocessing_config()

    ob = ImagePreprocessing(config_params)
    folder_path = ob.get_folder_path("data/raw") # raw folder 

    interim_path = ob.get_folder_path("data/interim")

    
    for folder in folder_path:
        # get image with path
        

    

    

2025-01-13 23:07:22,958 - root - INFO - Yaml read successfully from config/config.yaml
2025-01-13 23:07:22,961 - root - INFO - Yaml read successfully from params.yaml
2025-01-13 23:07:22,963 - root - ERROR - Exception occured while reading yaml file from                         location: params.yaml
 First argument must be mapping or iterable
2025-01-13 23:07:22,964 - root - INFO - .gitkeep not a directory
Removing .gitkeep
2025-01-13 23:07:22,967 - root - INFO - .DS_Store not a directory
Removing .DS_Store


{'data': {'google_drive_url': 'https://drive.google.com/file/d/10ApjOcTo6tjO34q5et-ij-tbrSm0Waqc/view?usp=drive_link', 'download_path': 'data/external/data.zip', 'extract_to': 'data/raw', 'interim_path': 'data/interim', 'processed_path': 'data/processed'}}
['.gitkeep', 'AB+', 'AB-', 'A+', 'B-', 'B+', 'A-', 'O-', 'O+']
['.DS_Store']


In [17]:
folder_path

['data/raw/AB+',
 'data/raw/AB-',
 'data/raw/A+',
 'data/raw/B-',
 'data/raw/B+',
 'data/raw/A-',
 'data/raw/O-',
 'data/raw/O+']

In [16]:
os.listdir('data/interim/AB-',)

['cluster_5_2567.BMP',
 'cluster_5_2201.BMP',
 'cluster_5_4664.BMP',
 'cluster_5_4102.BMP',
 'cluster_5_3645.BMP',
 'cluster_5_5234.BMP',
 'cluster_5_1052.BMP',
 'cluster_5_2229.BMP',
 'cluster_5_4880.BMP',
 'cluster_5_710.BMP',
 'cluster_5_1913.BMP',
 'cluster_5_248.BMP',
 'cluster_5_2771.BMP',
 'cluster_5_5750.BMP',
 'cluster_5_5618.BMP',
 'cluster_5_2605.BMP',
 'cluster_5_314.BMP',
 'cluster_5_5803.BMP',
 'cluster_5_3282.BMP',
 'cluster_5_4704.BMP',
 'cluster_5_670.BMP',
 'cluster_5_3057.BMP',
 'cluster_5_5432.BMP',
 'cluster_5_1867.BMP',
 'cluster_5_3902.BMP',
 'cluster_5_4937.BMP',
 'cluster_5_3080.BMP',
 'cluster_5_4089.BMP',
 'cluster_5_5396.BMP',
 'cluster_5_1872.BMP',
 'cluster_5_3903.BMP',
 'cluster_5_3724.BMP',
 'cluster_5_5427.BMP',
 'cluster_5_2348.BMP',
 'cluster_5_4711.BMP',
 'cluster_5_4063.BMP',
 'cluster_5_895.BMP',
 'cluster_5_2823.BMP',
 'cluster_5_1494.BMP',
 'cluster_5_5180.BMP',
 'cluster_5_1331.BMP',
 'cluster_5_467.BMP',
 'cluster_5_315.BMP',
 'cluster_5_4275.B

In [10]:
from pathlib import Path
Path("cluster_1_0.BMP").resolve()

PosixPath('/Users/goldyrana/mess/deep_learning/projects/blood_group_detection/cluster_1_0.BMP')